In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [9]:
# download a dataset
dataset = load_dataset("yelp_review_full")

Reusing dataset yelp_review_full (C:\Users\Oscar Pang\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00, 20.30it/s]


In [10]:
# tokenize the dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to C:\Users\Oscar Pang\.cache\huggingface\transformers\tmpg_0erz_w
storing https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json in cache at C:\Users\Oscar Pang/.cache\huggingface\transformers\ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for C:\Users\Oscar Pang/.cache\huggingface\transformers\ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
https://huggingface.co/bert-base-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to C:\Users\Oscar Pang\.cache\huggingface\transformers\tmp1k235jdh
storing https://huggingface.co/bert-base-cased/resolve/main/config.json in cache at C:\Users\Oscar Pang/.cache\huggingf

In [11]:
# create a smaller subset of the full dataset to fine-tune on to reduce the time it takes
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [12]:
from transformers import DefaultDataCollator

# the DefaultDataCollator assembles tensors into a batch for the model to train on
data_collator = DefaultDataCollator(return_tensors="tf")

In [13]:
# convert the tokenized datasets to TensorFlow datasets

tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [1]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.CategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=1)

c:\Users\Oscar Pang\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'tf_train_dataset' is not defined

In [ ]:
model.save("fine_tined.model")